In [1]:
import pandas as pd
import nltk
import numpy as np
import random
import re
from nltk.corpus import stopwords
stopword = list(set(stopwords.words('english')))

def clearstring(string):
    string = ' '.join(nltk.word_tokenize(string))
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return ' '.join([y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0 and y not in stopword])

In [2]:
df1 = pd.read_csv('Full-Economic-News-DFE-839861.csv',encoding = "ISO-8859-1")
sentiment1 = df1[['positivity', 'text']]
sentiment1 = sentiment1.dropna()
sentiment1.head()

,positivity,text
0,3.0,NEW YORK -- Yields on most certificates of dep...
4,3.0,NEW YORK -- Indecision marked the dollar's ton...
5,3.0,"Stocks declined, as investors weighed slower-t..."
9,4.0,The U.S. dollar declined against most major fo...
12,4.0,Author: James B. Stewart</br></br>The dreaded ...


In [3]:
sentiment1.positivity.loc[sentiment1.positivity < 5] = 0
sentiment1.positivity.loc[sentiment1.positivity >= 5] = 1

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
np.unique(sentiment1.positivity, return_counts=True)

(array([ 0.,  1.]), array([633, 787]))

In [5]:
df2 = pd.read_csv('us-economic-newspaper.csv',encoding = "ISO-8859-1")
sentiment2 = df2[['positivity', 'next_sentence','text']]
sentiment2 = sentiment2.dropna()
sentiment2.head()

,positivity,next_sentence,text
1,6.0,"The bond market continued to rally, propelling...","The Nasdaq composite index, home of technology..."
2,5.0,The Nasdaq market was stricken by the collapse...,"The bond market continued to rally, propelling..."
3,4.0,uring the marketÛªs split ÛÓ was 4.26 percen...,The Nasdaq market was stricken by the collapse...
4,3.0,". ""It's a nervous market,Û said Lawrence R. ...",uring the marketÛªs split ÛÓ was 4.26 percen...
5,4.0,"10,000\tpeople, pulled off a stun-ling economi...",Ore. ÛÓ Fishing and timber are not what they ...


In [6]:
sentiment2.iloc[0, 1]

"The bond market continued to rally, propelling (he normally placid ,' Dow Jones utility average to a gain of 6.01\tpoints, or 2.5,6 percent, to 240.61 \x89ÛÓ the best one-day.advance, in percentage terms, in nearly five years. The yield on the 30-year Treasury bond fell to another record low. (Page \x89Û¢ D18.J"

In [7]:
sentiment2.iloc[0, 2]

'The Nasdaq composite index, home of technology companies large and small, fell 1.7 percent even as the Dow Jones industrial average rose 20.81 points, to 3,342.99.'

In [8]:
sentiment2.positivity.loc[sentiment2.positivity < 5] = 0
sentiment2.positivity.loc[sentiment2.positivity >= 5] = 1

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
sentiment2.head()

,positivity,next_sentence,text
1,1.0,"The bond market continued to rally, propelling...","The Nasdaq composite index, home of technology..."
2,1.0,The Nasdaq market was stricken by the collapse...,"The bond market continued to rally, propelling..."
3,0.0,uring the marketÛªs split ÛÓ was 4.26 percen...,The Nasdaq market was stricken by the collapse...
4,0.0,". ""It's a nervous market,Û said Lawrence R. ...",uring the marketÛªs split ÛÓ was 4.26 percen...
5,0.0,"10,000\tpeople, pulled off a stun-ling economi...",Ore. ÛÓ Fishing and timber are not what they ...


In [10]:
results, texts = [], []
for i in range(sentiment1.shape[0]):
    splitted = sentiment1.iloc[i, 1].split('</br></br>')
    for k in splitted:
        texts.append(clearstring(k))
        results.append(sentiment1.iloc[i,0])

In [11]:
print(len(results))
print(len(texts))

7120
7120


In [12]:
results2, texts2 = [], []
for i in range(sentiment2.shape[0]):
    texts2.append(clearstring(sentiment2.iloc[i, 1]))
    texts2.append(clearstring(sentiment2.iloc[i, 2]))
    results2.append(sentiment2.iloc[i,0])
    results2.append(sentiment2.iloc[i,0])

In [13]:
print(len(results2))
print(len(texts2))

5798
5798


In [14]:
results = results +results2
texts = texts +texts2

In [15]:
combined = list(zip(results, texts))
random.shuffle(combined)
results[:], texts[:] = zip(*combined)

In [16]:
import pickle
results=np.array([results,texts]).T
with open('dataset.p', 'wb') as fopen:
    pickle.dump(results,fopen,protocol=2)

In [17]:
with open('dataset.p','rb') as fopen:
    results=pickle.load(fopen)